In [1]:
from corpus import ConllCorpusReaderX

In [2]:
import os
import re
from pymorphy2.tokenizers import simple_word_tokenize

result = []

for file in os.listdir('./bsnlp_dataset/eu_annot/'):
    number = int(file.split('.')[0])
    with open('./bsnlp_dataset/eu_annot/' + file, encoding='utf-8') as annot_file:
        annot = annot_file.readlines()[1:]
        annot = [simple_word_tokenize(el.strip()) for el in annot]
        annot = [el[:int((len(el) - 2)/2)] + [el[-2]] for el in annot]
    with open('./bsnlp_dataset/eu/file_' + str(number) + "_ru.txt", encoding='utf-8') as text_file:
        text = text_file.readlines()[4:]
        text = [simple_word_tokenize(line.strip()) for line in text]
        text = [el for el in text if el != []]
        total_words = []
        for el in text:
            total_words += el
            total_words += ' '
        total_words = [el.lower() for el in total_words]
        total_ne = ['O' for el in total_words]

    for ne in annot:
        words_count = len(ne) - 1
        for i in range(len(total_words) - words_count - 1):
            if ne[:-1] == total_words[i: i + words_count]:
                if words_count == 1:
                    total_ne[i] = 'S-' + ne[-1]
                else:
                    total_ne[i] = 'B-' + ne[-1]
                    total_ne[i + words_count - 1] = 'E-' + ne[-1]
                    for j in range(i + 1, i + words_count - 1):
                        total_ne[j] = 'I-' + ne[-1]

    for word, ne in zip(total_words, total_ne):
        if word != " ":
            result.append(word + ' ' +  ne + '\n')
        else:
            result.append('\n')
            
with open('./bsnlp_dataset/ec.txt', 'w+', encoding='utf-8') as file:
    file.write("-DOCSTART- O\n")
    for line in result:
        file.write(line)

In [3]:
result = []

for file in os.listdir('./bsnlp_dataset/trump_annot/'):
    number = int(file.split('.')[0])
    with open('./bsnlp_dataset/trump_annot/' + file, encoding='utf-8') as annot_file:
        annot = annot_file.readlines()[1:]
        annot = [simple_word_tokenize(el.strip()) for el in annot]
        annot = [el[:int((len(el) - 2)/2)] + [el[-2]] for el in annot]
    with open('./bsnlp_dataset/trump/file_' + str(number) + ".txt", encoding='utf-8') as text_file:
        text = text_file.readlines()[4:]
        text = [simple_word_tokenize(line.strip()) for line in text]
        text = [el for el in text if el != []]
        total_words = []
        for el in text:
            total_words += el
            total_words += ' '
        total_words = [el.lower() for el in total_words]
        total_ne = ['O' for el in total_words]

    for ne in annot:
        words_count = len(ne) - 1
        for i in range(len(total_words) - words_count - 1):
            if ne[:-1] == total_words[i: i + words_count]:
                if words_count == 1:
                    total_ne[i] = 'S-' + ne[-1]
                else:
                    total_ne[i] = 'B-' + ne[-1]
                    total_ne[i + words_count - 1] = 'E-' + ne[-1]
                    for j in range(i + 1, i + words_count - 1):
                        total_ne[j] = 'I-' + ne[-1]

    for word, ne in zip(total_words, total_ne):
        if word != " ":
            result.append(word + ' ' +  ne + '\n')
        else:
            result.append('\n')
            
with open('./bsnlp_dataset/trump.txt', 'w+', encoding='utf-8') as file:
    file.write("-DOCSTART- O\n")
    for line in result:
        file.write(line)

In [4]:
trump_dataset = ConllCorpusReaderX('./bsnlp_dataset/', 
                              fileids='trump.txt', 
                              columntypes=('words', 'ne'))

In [5]:
eu_dataset = ConllCorpusReaderX('./bsnlp_dataset/', 
                              fileids='eu.txt', 
                              columntypes=('words', 'ne'))

In [6]:
print(trump_dataset.get_ne()[:10])

[('трамп', 'S-PER'), ('разошелся', 'O'), ('во', 'O'), ('мнении', 'O'), ('с', 'O'), ('мишель', 'B-PER'), ('обамой', 'E-PER'), ('о', 'O'), ('судьбе', 'O'), ('сша', 'S-LOC')]


In [7]:
print(trump_dataset.words())
print(len(trump_dataset.words()))

['трамп', 'разошелся', 'во', 'мнении', 'с', 'мишель', ...]
5474


In [12]:
import numpy as np
from generator import Generator

words = [[el] for el in trump_dataset.words()[:4000]]
gen = Generator(column_types=['word'], context_len=1)
X = gen.generate(words, path='./trump_dataset.npy')
y = np.array([el[1] for el in trump_dataset.get_ne()[:4000]])

In [13]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

In [14]:
clf.fit(X[:3000], y[:3000])
y_pred = clf.predict(X[3000:4000])

In [15]:
right = 0
total_a = 0
total_b = 0
for a, b in zip(y_pred, y[3000:4000]):
    if a != 'O':
        total_a += 1
    if b != 'O':
        total_b += 1    
    if a != 'O' and a == b:
        right += 1
accuracy = right/total_a
recall = right/total_b
f = 2 * accuracy * recall /(accuracy + recall)
print(f)

0.61
